# Jax profiler test: marginal_util_and_exp_max_value_states_period

In [1]:
import io
import pickle
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
#from toy_models.consumption_retirement_model import (compute_expected_value, compute_next_period_marginal_utility,inverse_marginal_utility_crra, utility_func_crra)
from toy_models.consumption_retirement_model.utility_functions import utility_func_crra, marginal_utility_crra, inverse_marginal_utility_crra
from toy_models.consumption_retirement_model.budget_functions import budget_constraint
from toy_models.consumption_retirement_model.exogenous_processes import get_transition_matrix_by_state
from dcegm.pre_processing import get_partial_functions, params_todict
from dcegm.marg_utilities_and_exp_value import marginal_util_and_exp_max_value_states_period
import jax
import jax.numpy as jnp
from jax import profiler
import os
from dcegm.solve import solve_dcegm

## Specify *utility functions*, *params* and *options*

In [2]:
utility_functions = {
    "utility": utility_func_crra,
    "inverse_marginal_utility": inverse_marginal_utility_crra,
    "marginal_utility": marginal_utility_crra,
}

In [3]:
params = """category,name,value,comment
beta,beta,0.95,discount factor
delta,delta,0,disutility of work
utility_function,theta,1,CRRA coefficient
wage,constant,0.75,age-independent labor income
wage,exp,0.04,return to experience
wage,exp_squared,-0.0004,return to experience squared
shocks,sigma,0.25,shock on labor income sigma parameter/standard deviation
shocks,lambda,2.220400e-16,taste shock (scale) parameter
assets,interest_rate,0.05,interest rate on capital
assets,initial_wealth_low,0,lowest level of initial wealth (relevant for simulation)
assets,initial_wealth_high,30,highest level of initial wealth (relevant for simulation)
assets,max_wealth,75,maximum level of wealth
assets,consumption_floor,0.0,consumption floor/retirement safety net (only relevant in the dc-egm retirement model)
"""

In [4]:
params = pd.read_csv(io.StringIO(params), index_col=["category", "name"])
params

value  \
category         name                                
beta             beta                 9.500000e-01   
delta            delta                0.000000e+00   
utility_function theta                1.000000e+00   
wage             constant             7.500000e-01   
                 exp                  4.000000e-02   
                 exp_squared         -4.000000e-04   
shocks           sigma                2.500000e-01   
                 lambda               2.220400e-16   
assets           interest_rate        5.000000e-02   
                 initial_wealth_low   0.000000e+00   
                 initial_wealth_high  3.000000e+01   
                 max_wealth           7.500000e+01   
                 consumption_floor    0.000000e+00   

                                                                                comment  
category         name                                                                    
beta             beta                                                   discount factor  
delta            delta                                               disutility of work  
utility_function theta                                                 CRRA coefficient  
wage             constant                                  age-independent labor income  
                 exp                                               return to experience  
                 exp_squared                               return to experience squared  
shocks           sigma                shock on labor income sigma parameter/standard...  
                 lambda                                   taste shock (scale) parameter  
assets           interest_rate                                 interest rate on capital  
                 initial_wealth_low   lowest level of initial wealth (relevant for s...  
                 initial_wealth_high  highest level of initial wealth (relevant for ...  
                 max_wealth                                     maximum level of wealth  
                 consumption_floor    consumption floor/retirement safety net (only ...

In [5]:
params_dict = params_todict(params)

In [6]:
options = """n_periods: 25
min_age: 20
n_discrete_choices: 1
grid_points_wealth: 100
quadrature_points_stochastic: 10
n_simulations: 10
n_exog_processes: 1
"""

In [7]:
options = yaml.safe_load(options)
options

{'n_periods': 25,
 'min_age': 20,
 'n_discrete_choices': 1,
 'grid_points_wealth': 100,
 'quadrature_points_stochastic': 10,
 'n_simulations': 10,
 'n_exog_processes': 1}

## Specify inputs for function **marginal_util_and_exp_max_value_states_period**

In [8]:
(
    compute_utility,
    compute_marginal_utility,
    compute_inverse_marginal_utility,
    compute_value,
    compute_next_period_wealth,
    transition_function,
) = get_partial_functions(
    params_dict,
    options,
    utility_functions,
    budget_constraint,
    get_transition_matrix_by_state,
)

In [9]:
taste_shock_scale = 2.220400e-16
exogenous_savings_grid = pickle.load(open("profiling_resources/exogenous_savings_grid.pkl", "rb"))
income_shock_draws = pickle.load(open("profiling_resources/income_shock_draws.pkl", "rb"))
income_shock_weights = pickle.load(open("profiling_resources/income_shock_weights.pkl", "rb"))
possible_child_states = pickle.load(open("profiling_resources/possible_child_states.pkl", "rb"))
choices_child_states = pickle.load(open("profiling_resources/choices_child_states.pkl", "rb"))
policies_child_states = pickle.load(open("profiling_resources/policies_child_states.pkl", "rb"))
values_child_states = pickle.load(open("profiling_resources/values_child_states.pkl", "rb"))

In [10]:
num_states = 5000
states_repeated = jnp.repeat(possible_child_states, num_states, axis=0)
choices_repeated = jnp.repeat(choices_child_states, num_states, axis=0)
policies_repeated = jnp.repeat(policies_child_states, num_states, axis=0)
values_repeated = jnp.repeat(values_child_states, num_states, axis=0)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [11]:
num_savings_repeats = 5
savings_repeated = jnp.repeat(exogenous_savings_grid, num_savings_repeats, axis=0)

In [12]:
with jax.profiler.trace("jax-trace", create_perfetto_link=True):
    marginal_util, max_exp_value = marginal_util_and_exp_max_value_states_period(
        compute_next_period_wealth,
        compute_marginal_utility,
        compute_value,
        taste_shock_scale,
        savings_repeated,
        income_shock_draws,
        income_shock_weights,
        states_repeated,
        choices_repeated,
        policies_repeated,
        values_repeated,
    )

2023-02-14 23:37:31.759811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-14 23:37:32.432385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-14 23:37:32.432470: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-14 23:37:32.432478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [12]:
%%timeit
marginal_util, max_exp_value = marginal_util_and_exp_max_value_states_period(
    compute_next_period_wealth,
    compute_marginal_utility,
    compute_value,
    taste_shock_scale,
    savings_repeated,
    income_shock_draws,
    income_shock_weights,
    states_repeated,
    choices_repeated,
    policies_repeated,
    values_repeated,
)

3.23 s ± 186 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# use jax profiler
def profiled_marginal_util_and_exp_max_value_states_period(
    compute_next_period_wealth,
    compute_marginal_utility,
    compute_value,
    taste_shock_scale,
    exogenous_savings_grid,
    income_shock_draws,
    income_shock_weights,
    possible_child_states,
    choices_child_states,
    policies_child_states,
    values_child_states,
):
    return profiler.call(
        marginal_util_and_exp_max_value_states_period,
        compute_next_period_wealth,
        compute_marginal_utility,
        compute_value,
        taste_shock_scale,
        exogenous_savings_grid,
        income_shock_draws,
        income_shock_weights,
        possible_child_states,
        choices_child_states,
        policies_child_states,
        values_child_states,
    )